In [92]:
# Load data

import pandas as pd

df = pd.DataFrame.from_csv('episodes.tsv', sep='\t', index_col=False, encoding='utf-8')    

In [93]:
# Extract season and episodes out of all three fields

df.comments_season = df.comments_ep.str.split('e').str[0]
df.comments_season = df.comments_season.str.split('s').str[1].astype(float)
df.comments_episode = df.comments_ep.str.split('e').str[1].astype(float)

df.title_season = df.title_ep.str.split('e').str[0]
df.title_season = df.title_season.str.split('s').str[1].astype(float)
df.title_episode = df.title_ep.str.split('e').str[1].astype(float)

df.flair_season = df.flair_text.str.split('(?i)e').str[0]
df.flair_season = df.flair_season.str.split('(?i)s').str[1].apply(pd.to_numeric, errors='coerce')
df.flair_episode = df.flair_text.str.split('(?i)e').str[1].apply(pd.to_numeric, errors='coerce')

In [154]:
# Some basic data analysis and stats on our dataset. 
# A big goal here is to find the best way to find the correct episode

num = len(df)
num_es_agree = sum((df.title_season == df.comments_season) & (df.title_episode == df.comments_episode))
num_tags = sum((df.flair_season > 0) & (df.flair_episode > 0))
num_tags_title = sum((df.title_episode == df.flair_episode) & (df.title_season == df.flair_season))
num_tags_comments = sum((df.comments_episode == df.flair_episode) & (df.comments_season == df.flair_season))
num_tags_es_agree = sum(
                        ((df.title_season == df.comments_season) & (df.title_episode == df.comments_episode)) & 
                        ((df.flair_season > 0) & (df.flair_episode > 0))
                       )
num_est_agree = sum(
                    ((df.title_season == df.comments_season) & (df.title_episode == df.comments_episode)) & 
                    (df.title_episode == df.flair_episode) &
                    (df.title_season == df.flair_season)
                   )

print("Number of rows: {0}".format(num))
print("Number of rows where title and comments episodes agree with each other: {0}".format(num_es_agree))
print("Number of rows with a tag (from reddit flair): {0}".format(num_tags))
print("Number of rows where title agrees with tag: {0}".format(num_tags_title))
print("Number of rows where comments agrees with tag: {0}".format(num_tags_comments))
print("Number of rows where title and comments episodes agree with each other and there's a tag: {0}".format(
        num_tags_es_agree
     ))       
print("Number of rows where title and comments episodes and flair tags agree with each other: {0}".format(
        num_est_agree
     ))

print("""
Some take-aways from this:
* We processed a total of {num} submissions
* Out of {num_tags} tagged/flaired submissions, {num_tags_title} submissions match using title ({num_tags_title_pct}%).
* Out of {num_tags} tagged/flaired submissions, {num_tags_comments} submissions match using comments ({num_tags_comments_pct}%).
* Out of {num_tags_es_agree} tagged submissions where the title and comments tags agree, 
    {num_est_agree} submissions match using both comments and episodes ({pct_est_agree}%).
* We have {num_es_agree} submissions to work with if we require the title and comments ep to match.


""".format(**{
            'num': num,
            'num_es_agree': num_es_agree,
            'num_tags': num_tags,
            'num_tags_title': num_tags_title,
            'num_tags_comments': num_tags_comments,
            'num_tags_title_pct': "{0:.2f}".format(100.0 * num_tags_title / num_tags),
            'num_tags_comments_pct': "{0:.2f}".format(100.0 * num_tags_comments / num_tags),
            'num_tags_es_agree': num_tags_es_agree,
            'num_est_agree': num_est_agree,
            'pct_est_agree': "{0:.2f}".format(100.0 * num_est_agree / num_tags_es_agree),
             }
          )
     )

Number of rows: 65500
Number of rows where title and comments episodes agree with each other: 14106
Number of rows with a tag (from reddit flair): 13972
Number of rows where title agrees with tag: 7443
Number of rows where comments agrees with tag: 6345
Number of rows where title and comments episodes agree with each other and there's a tag: 4185
Number of rows where title and comments episodes and flair tags agree with each other: 3975

Some take-aways from this:
* We processed a total of 65500 submissions
* Out of 13972 tagged/flaired submissions, 7443 submissions match using title (53.27%).
* Out of 13972 tagged/flaired submissions, 6345 submissions match using comments (45.41%).
* Out of 4185 tagged submissions where the title and comments tags agree, 
    3975 submissions match using both comments and episodes (94.98%).
* We have 14106 submissions to work with if we require the title and comments ep to match.





In [156]:
# Looking at submissions where both comments and episodes agree gives a decent 95.00% match rate
# Let's look at some examples where this fails - where we have a tag, and title and comments matches, but tag doesn't
pd.set_option('display.max_colwidth', 500)
df[(
    ((df.flair_season > 0) & (df.flair_episode > 0)) &
    ((df.title_season == df.comments_season) & (df.title_episode == df.comments_episode)) &
    ((df.title_episode != df.flair_episode) | (df.title_season != df.flair_season))
)][['submission_permalink', 'comments_ep', 'flair_text']]

# Looking at the first 10 submissions, 3/10 were tagged incorrectly. Assuming this is consistent throughout, 
# the actual correct match rate is closer to 95% + (5% * 30%) = 96.5%

,submission_permalink,comments_ep,flair_text
264,https://www.reddit.com/r/TheSimpsons/comments/4sri37/moes_slow_descent_with_the_lie_detector_is_by_far/,s08e10,s07e01
284,https://www.reddit.com/r/TheSimpsons/comments/4t8bum/of_course_one_thing_mother_blue_jay_cant_defend/,s10e03,s10e08
390,https://www.reddit.com/r/TheSimpsons/comments/4t2150/lets_just_say_this_he_spelled_yale_with_a_6/,s08e04,s08e06
531,https://www.reddit.com/r/TheSimpsons/comments/4rafon/skinner_wasnt_always_a_weenie/,s09e02,s04e04
839,https://www.reddit.com/r/TheSimpsons/comments/4q7bel/oh_you_think_this_is_funny_ill_tell_you_something/,s25e11,s08e07
866,https://www.reddit.com/r/TheSimpsons/comments/4qvhvc/otto_loves_to_get_blotto/,s07e12,s15e15
1304,https://www.reddit.com/r/TheSimpsons/comments/4p16lu/come_and_lick_me_big_boyi_said_lick_me/,s26e02,s18e7
1400,https://www.reddit.com/r/TheSimpsons/comments/4oaq10/the_year_was_1968_we_were_on_recon_in_a_steaming/,s07e12,s07e13
1558,https://www.reddit.com/r/TheSimpsons/comments/4o1x6e/step_on_it_mom_dads_signaling_that_he_wants_to_go/,s13e17,s10e05
1737,https://www.reddit.com/r/TheSimpsons/comments/4n0olu/ca_primary_today_everyones_bragging_about_who/,s19e10,s08e01


In [167]:
# look at some summary stats of seasons
df_analyze = df[(df.title_season == df.comments_season) & (df.title_episode == df.comments_episode)]
df_analyze.title_season

AttributeError: 'DataFrame' object has no attribute 'title_season'